In [22]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
%cd '/content/drive/MyDrive/CS 5228 Project/Team Intelligent Realtors'

/content/drive/MyDrive/CS 5228 Project/Team Intelligent Realtors


In [24]:
pip install catboost


# Ensemble Modelling


Stacked Ensembling. Held out a test set from the training set and use K fold cross validation to train cat boost, XGB and linear regression on top of the results from the two classifiers

In [44]:
%%time
from sklearn.linear_model import LinearRegression
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgbRegressor



data = pd.read_csv('final_train.csv')

data_Set = data[['resale_price','region', 'floor_area_sqm', 'latitude', 'longitude', 'lease_commence_date', 'min_mrt_dist', 'cbd_dist', 'min_hawk_dist', 'min_com_dist', 'min_mall_dist', 'mrt', 'adult_Perc', 'sale_month', 'sale_year', 'storey_mid', 'lease_period', 'flat_type_ord','flat_model_ord', 'mature_town']]

col = data_Set.columns.tolist()

data_Set = data_Set.reindex(columns = col)

Y = data_Set.iloc[:,0]
X = data_Set.iloc[:, 1:len(col)]


### Standardization
num_data = X.select_dtypes(include=np.number)
cat_data = X.select_dtypes(exclude=np.number)
x_scaler = StandardScaler()
num_scaled = x_scaler.fit_transform(num_data)
num_scaled = pd.DataFrame(num_scaled,columns=num_data.columns)
X = pd.concat([num_scaled.reset_index(drop=True),cat_data.reset_index(drop=True)],axis=1)

X_train, X_test, y_train, y_test = train_test_split( X, Y, train_size = 0.9, stratify = X['region'],random_state = 50)
y_train = y_train.squeeze()
y_test = y_test.squeeze()

kf = KFold(n_splits=5)
rmse1 = []
rmse2 = []
xgb_rmse = []
catboost_rmse = []
linear_rmse = []

for train_index, val_index in kf.split(X_train):
    # Create the next combination of training and validation folds
    X_train_subset, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_subset, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
    y_train_subset = y_train_subset.squeeze()
    y_val = y_val.squeeze()
    y_val = np.sqrt(y_val)
    y_train_subset = np.sqrt(y_train_subset)

    categorical_features_indices = np.where(X_train_subset.dtypes != np.float)[0]
    print(categorical_features_indices)
    clf_cat=CatBoostRegressor(learning_rate=0.4,depth=8).fit(X_train_subset,y_train_subset,cat_features=categorical_features_indices,eval_set=(X_val, y_val))
    y_pred1 = clf_cat.predict(X_val)

    val1 = np.sqrt(mean_squared_error(y_val**2, y_pred1**2))
    rmse1.append(int(val1))

    clf_xgb = xgbRegressor.XGBRegressor(max_depth=40,criterion='mse',min_samples_split=25,random_state=0).fit(X_train_subset, y_train_subset)

    y_pred2 = clf_xgb.predict(X_val)
    val2 = np.sqrt(mean_squared_error(y_val**2, y_pred2**2))
    rmse2.append(int(val2))

    df = pd.concat([ pd.Series(y_pred1**2), pd.Series(y_pred2**2)], axis=1)
    model = LinearRegression()
    model.fit(df,y_val**2)

    y_catboost = clf_cat.predict(X_test)
    catboost = np.sqrt(mean_squared_error(y_test,y_catboost**2))
    catboost_rmse.append(int(catboost))

    y_xgb = clf_xgb.predict(X_test)
    xgb = np.sqrt(mean_squared_error(y_test,y_xgb**2))
    xgb_rmse.append(int(xgb))

    df = pd.concat([ pd.Series(y_catboost**2), pd.Series(y_xgb**2)], axis=1)
    y_predict = model.predict(df)
    rmse = np.sqrt(mean_squared_error(y_test,y_predict))
    linear_rmse.append(int(rmse))

  
    print('RMSE on validation sets of Catboost and XGB are: ',rmse1,rmse2)
    print('RMSE on test sets of Catboost and XGB are: ',catboost_rmse,xgb_rmse)
    print('RMSE on test set from Linear Regression is: ',linear_rmse)




Streaming output truncated to the last 5000 lines.
51:	learn: 18.0052902	test: 18.2527220	best: 18.2527220 (51)	total: 4s	remaining: 1m 12s
52:	learn: 17.9402892	test: 18.1878717	best: 18.1878717 (52)	total: 4.08s	remaining: 1m 13s
53:	learn: 17.8786389	test: 18.1283417	best: 18.1283417 (53)	total: 4.17s	remaining: 1m 13s
54:	learn: 17.8197181	test: 18.0797398	best: 18.0797398 (54)	total: 4.26s	remaining: 1m 13s
55:	learn: 17.7196960	test: 17.9839174	best: 17.9839174 (55)	total: 4.34s	remaining: 1m 13s
56:	learn: 17.6640497	test: 17.9368512	best: 17.9368512 (56)	total: 4.42s	remaining: 1m 13s
57:	learn: 17.6101507	test: 17.8850503	best: 17.8850503 (57)	total: 4.5s	remaining: 1m 13s
58:	learn: 17.5535927	test: 17.8303424	best: 17.8303424 (58)	total: 4.58s	remaining: 1m 13s
59:	learn: 17.5041360	test: 17.7963187	best: 17.7963187 (59)	total: 4.67s	remaining: 1m 13s
60:	learn: 17.4648167	test: 17.7554105	best: 17.7554105 (60)	total: 4.78s	remaining: 1m 13s
61:	learn: 17.3982526	test: 17.68

In [13]:
print('RMSE on validation sets of Catboost and XGBoost are: ',rmse1,rmse2)
print('RMSE on test sets of Catboost and XGBoost are: ',catboost_rmse,xgb_rmse)
print('RMSE on test set from Linear Regression is: ',linear_rmse)

RMSE on validation sets of Catboost and XGBoost are:  [16444, 16408, 16313, 16395, 16341] [17797, 17758, 17853, 17838, 17697]
RMSE on test sets of Catboost and XGBoost are:  [16658, 16677, 16572, 16580, 16581] [17912, 17956, 17992, 17903, 17936]
RMSE on test set from Linear Regression is:  [16345, 16388, 16334, 16318, 16327]


In [16]:
np.mean(linear_rmse)

16342.4

Final Training

In [32]:
%%time
from sklearn.linear_model import LinearRegression
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgbRegressor


data = pd.read_csv('final_train.csv')

data_Set = data[['resale_price','region', 'floor_area_sqm', 'latitude', 'longitude', 'lease_commence_date', 'min_mrt_dist', 'cbd_dist', 'min_hawk_dist', 'min_com_dist', 'min_mall_dist', 'mrt', 'adult_Perc', 'sale_month', 'sale_year', 'storey_mid', 'lease_period', 'flat_type_ord','flat_model_ord', 'mature_town']]

col = data_Set.columns.tolist()

data_Set = data_Set.reindex(columns = col)

Y = data_Set.iloc[:,0]
X = data_Set.iloc[:, 1:len(col)]

### Standardization
num_data = X.select_dtypes(include=np.number)
cat_data = X.select_dtypes(exclude=np.number)
x_scaler = StandardScaler()
num_scaled = x_scaler.fit_transform(num_data)
num_scaled = pd.DataFrame(num_scaled,columns=num_data.columns)
X = pd.concat([num_scaled.reset_index(drop=True),cat_data.reset_index(drop=True)],axis=1)

X_train, X_test, y_train, y_test = train_test_split( X, Y, train_size = 0.8, stratify = X['region'],random_state = 50)
y_train = y_train.squeeze()
y_test = y_test.squeeze()
y_test = np.sqrt(y_test)
y_train = np.sqrt(y_train)

categorical_features_indices = np.where(X_train.dtypes != np.float)[0]
print(categorical_features_indices)
clf_cat=CatBoostRegressor(learning_rate=0.4,depth=8).fit(X_train,y_train,cat_features=categorical_features_indices,eval_set=(X_test, y_test))
clf_xgb = xgbRegressor.XGBRegressor(max_depth=40,criterion='mse',min_samples_split=25,random_state=0).fit(X_train, y_train)

y_pred1 = clf_cat.predict(X_test)

y_pred2 = clf_xgb.predict(X_test)


rmse1 = np.sqrt(mean_squared_error(y_test**2, y_pred1**2))
print('Catboost rsme {:.3f}'.format(rmse1))

rmse2 = np.sqrt(mean_squared_error(y_test**2,y_pred2**2))
print('XGBoost rsme {:.3f}'.format(rmse2))


df = pd.concat([pd.Series(y_pred1**2), pd.Series(y_pred2**2)], axis=1)
model = LinearRegression()
model.fit(df,y_test**2)



[]
0:	learn: 77.6976360	test: 77.6933949	best: 77.6933949 (0)	total: 153ms	remaining: 2m 32s
1:	learn: 55.8638127	test: 55.7054961	best: 55.7054961 (1)	total: 237ms	remaining: 1m 58s
2:	learn: 44.0339260	test: 43.9046442	best: 43.9046442 (2)	total: 319ms	remaining: 1m 46s
3:	learn: 37.7566088	test: 37.5949449	best: 37.5949449 (3)	total: 404ms	remaining: 1m 40s
4:	learn: 33.3843624	test: 33.2879987	best: 33.2879987 (4)	total: 486ms	remaining: 1m 36s
5:	learn: 30.9354791	test: 30.8577676	best: 30.8577676 (5)	total: 567ms	remaining: 1m 33s
6:	learn: 29.2930557	test: 29.2302882	best: 29.2302882 (6)	total: 648ms	remaining: 1m 31s
7:	learn: 28.2200174	test: 28.1734014	best: 28.1734014 (7)	total: 726ms	remaining: 1m 30s
8:	learn: 27.2226595	test: 27.1893529	best: 27.1893529 (8)	total: 807ms	remaining: 1m 28s
9:	learn: 26.3626922	test: 26.3423657	best: 26.3423657 (9)	total: 892ms	remaining: 1m 28s
10:	learn: 25.6863364	test: 25.6576303	best: 25.6576303 (10)	total: 966ms	remaining: 1m 26s
11:	l

In [37]:
test_data=pd.read_csv('final_test.csv')

test = test_data[['region', 'floor_area_sqm', 'latitude', 'longitude', 'lease_commence_date', 'min_mrt_dist', 'cbd_dist', 'min_hawk_dist', 'min_com_dist', 'min_mall_dist', 'mrt', 'adult_Perc', 'sale_month', 'sale_year', 'storey_mid', 'lease_period', 'flat_type_ord','flat_model_ord', 'mature_town']]

### Standardization
num_data = test.select_dtypes(include=np.number)
cat_data = test.select_dtypes(exclude=np.number)
num_scaled = x_scaler.transform(num_data)
num_scaled = pd.DataFrame(num_scaled,columns=num_data.columns)
test = pd.concat([num_scaled.reset_index(drop=True),cat_data.reset_index(drop=True)],axis=1)


y_pred1 = clf_cat.predict(test)

y_pred2 = clf_xgb.predict(test)

df = pd.concat([pd.Series(y_pred1**2), pd.Series(y_pred2**2)], axis=1)

final_y_pred = model.predict(df)

final=pd.DataFrame(final_y_pred,columns=['Predicted'])

final['ID']=final.index

final=final[['ID','Predicted']]

final.to_csv('output.csv',index=False)


In [38]:
data = pd.read_csv('output.csv')
print(data.mean())
data.head()

ID            53966.500000
Predicted    301487.101304
dtype: float64


,ID,Predicted
0,0,195959.527766
1,1,304072.629205
2,2,123328.532566
3,3,322661.199014
4,4,309804.687845
